In [ ]:
from boardlaw.grid import *

In [ ]:
for width in [1, 2, 4, 8]:
    for depth in [1, 2, 4, 8]:
        params = dict(width=width, depth=depth, boardsize=5, timelimit=45*60, desc="main/5")
        jittens.jobs.submit(
            cmd='python -c "from boardlaw.main import *; run_jittens()" >logs.txt 2>&1',
            dir='.',
            resources={'gpu': 1},
            params=params)

In [ ]:
from IPython import display

vast.jittenate(local=True)
while not jittens.finished():
    display.clear_output(wait=True)
    jittens.refresh()
    time.sleep(1)

In [ ]:
from vast import *
offers('gpu_name == "RTX 2080 Ti"')

In [ ]:
import jittens
jittens.jobs.jobs()